In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def fetch_crypto_data(symbol, start_date, end_date, filename):
    data = yf.download(symbol, start=start_date, end=end_date, interval='1d')
    data.to_csv(filename)
    print(f"Data for {symbol} saved to {filename}")

# Define parameters
crypto_symbols = {'BTC-USD': 'btc_data.csv', 'ETH-USD': 'eth_data.csv', 'XRP-USD': 'xrp_data.csv'}
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=6*365)).strftime('%Y-%m-%d')

# Fetch and save data
for symbol, filename in crypto_symbols.items():
    fetch_crypto_data(symbol, start_date, end_date, filename)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Data for BTC-USD saved to btc_data.csv


[*********************100%***********************]  1 of 1 completed


Data for ETH-USD saved to eth_data.csv


[*********************100%***********************]  1 of 1 completed

Data for XRP-USD saved to xrp_data.csv


In [4]:
import pandas as pd
import numpy as np

def compute_indicators(df):
    # Convert relevant columns to numeric
    df[['Close', 'High', 'Low', 'Open', 'Volume']] = df[['Close', 'High', 'Low', 'Open', 'Volume']].apply(pd.to_numeric, errors='coerce')

    # Drop any remaining non-numeric rows
    df = df.dropna()

    # Simple Moving Averages (SMA)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # Exponential Moving Average (EMA)
    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # MACD and Signal Line
    df['MACD'] = df['Close'].ewm(span=26, adjust=False).mean() - df['Close'].ewm(span=12, adjust=False).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    return df

def process_crypto_data(filename):
    # Read CSV, skipping first two rows
    df = pd.read_csv(filename, skiprows=2)

    # Rename columns correctly
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')

    # Compute indicators
    df = compute_indicators(df)

    # Save processed file
    processed_filename = f"processed_{filename}"
    df.to_csv(processed_filename, index=False)
    print(f"Processed data saved to {processed_filename}")

# Process all three datasets
for crypto in ['btc_data.csv', 'eth_data.csv', 'xrp_data.csv']:
    process_crypto_data(crypto)


Processed data saved to processed_btc_data.csv
Processed data saved to processed_eth_data.csv
Processed data saved to processed_xrp_data.csv
